# Flickbike

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## Importing the data
Allereerst wordt de data ingeladen.

In [ ]:
df = pd.read_csv('../input/bikes.csv', index_col=0, parse_dates=True)
df.head()

## Analysing the data

In [ ]:
df.dtypes

Omzetten naar correcte datatypes

In [ ]:
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])
df['date'] = pd.to_datetime(df['date'])

# extracting extra features
df['weekday'] = df['date'].dt.weekday_name
df['hour'] = df['date'].dt.hour

df.dtypes

In [ ]:
df.isna().sum()

## Verdiepingsopdracht

### Vraag 1
**Produceer een plot van verhuur-per-dag voor deze dataset.**  
  
Om dit voor elkaar te krijgen worden eerst alle dagen bij elkaar gegroupeerd zodat de hoeveelheid verhuren per dag opgeteld zullen worden.  
Hierna wordt deze dataset geplot.

In [ ]:
df_dates_count = df.groupby(df['date'].dt.date)['tripid'].count()

x = df_dates_count.index
y = df_dates_count.values

sns.set_style("darkgrid")
plt.plot(x, y)

plt.title("")
plt.xlabel("")
plt.ylabel("")

plt.figure(figsize=(18,18))
plt.show()

In [ ]:
df_dates_count.index = pd.to_datetime(df_dates_count.index)
df_three_months = df_dates_count.loc["2019-09-01":"2019-12-31"]

sns.set_style("darkgrid")
plt.plot(df_three_months.index, df_three_months.values)

plt.title("")
plt.xlabel("")
plt.ylabel("")

plt.xticks(rotation=45)
plt.figure(figsize=(18,18))
plt.show()

### Vraag 2
**Wat is de periodiciteit van deze dataset?**  
  
Om de vraag wat betreft periodiciteit te beantwoorden zullen we moeten gaan kijken of de data zich volgens een bepaald patroon gedraagt. Om deze vraag te beantwoorden bouwen we eerst een autocorrelatie plot.


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(df_dates_count.values, lags=30)
plt.show()

## Opleveropdracht

Voor de 



Deze terugkerende patronenen zal ik bij de volgende scopes gaan bekijken:  
- De dag van de week
- Uren van de dag
- Het weer
- Bijzondere dagen, zoals feestdagen, schoolvakanties, etc  
  
### De dag van de week